# A String Matching Story

#### 1. Imports

In [1]:
import itertools
import numpy as np
import pandas as pd

from nltk.metrics.distance import edit_distance
from fuzzywuzzy import fuzz
from polyfuzz import PolyFuzz
from polyfuzz.models import TFIDF

#### 2. The data

In [2]:
def data_generator(variant,flavour,product,quantity):
    permutations=itertools.product(variant,flavour,product,quantity)
    combinations=list(permutations)
    data=pd.DataFrame(combinations, columns=["variant","flavour","product","quantity"])
    data["product_option_1"]="beverages "+data["product"]+" "+data["variant"]+" "+data["flavour"]+" "+data["quantity"]
    data["product_option_2"]="beverage "+data["variant"]+" "+data["flavour"]+" "+data["product"]+" "+data["quantity"]
    results=pd.concat([data["product_option_1"],data["product_option_2"]])
    results.drop_duplicates(inplace=True)
    results=pd.DataFrame(results,columns=["product"])
    return(results)

In [3]:
external_data=data_generator(variant=["diet","dit","diiet",""],
                             flavour=["vanilla","chocolate","watermelon"],
                             product=["coke","cokke","spryte","sprite","prite","fanta","afnta","7up","seven up","sevenup","7-up","pepsi"],
                             quantity=["500ml","1.5L","2.25L","354ml","500cc","354cc",
                                       "500 ml","1.5 L","2.25 L","354 ml","500 cc","354 cc",
                                       "500 ml.","1.5 LT.","2.25 L","354 ml","500 cc","354 cc",
                                       "500 ml.","1.5 l.","2.25 l","354 ml.","500 cc.","354 cc",
                                       "500 ml","1.5 lts","2.25 lts","354 ml","500 cc","354 cc",
                                       "500 ml.","1.5 lts.","2.25 lts","354 ml","500 cc","354 cc",
                                       "500 ml.","1.5 lt.","2.25 lt","354 ml.","500 cc.","354 cc"])

In [4]:
# Beverages of internal source
variant=["DIET"]
product=["COKE","SPRITE","FANTA","7 UP","PEPSI"]
flavour=["VANILLA","CHOCOLATE","COCO","STRAWBERRY"]
quantity=["X 500ML","X 1.5L","X 2.25L","X 354ML"]
permutations=itertools.product(variant,flavour,product,quantity)
combinations=list(permutations)
internal_data=pd.DataFrame(combinations, columns=["variant","flavour","product","quantity"])
internal_data["product_name"]=internal_data["variant"]+" "+internal_data["flavour"]+" "+internal_data["product"]+" "+internal_data["quantity"]
internal_data=pd.DataFrame(internal_data["product_name"])

#### 3. String preprocessing

In [5]:
def string_processing(df:pd.DataFrame,ignored_columns:list):
    for col in df.columns.difference(ignored_columns):
        
        #Convert strings to lowercase
        df[col] = df[col].str.lower()
        
        #Remove the "-" symbol
        
        df[col] = df[col].str.replace(r"(\-)","",regex=True)

        # Remove words that don't add information
        df[col] = df[col].str.replace(r"(beverages|beverage)","",regex=True)
        
        # Manually correct the spelling of some products
        df[col] = df[col].str.replace(r"(seven up|sevenup)","7up",regex=True)
        df[col] = df[col].str.replace(r"(7 up)","7up",regex=True)
        
        # Replace consecutive blank spaces for 1 blank space
        df[col] = df[col].str.replace(r"(  |   )", " ",regex=True)
        df[col] = df[col].str.replace(r"(  )", " ",regex=True)
        
        # Remove starting blank space
        df[col] = df[col].str.replace(r"(^ )", "",regex=True)
        
        # Remove blank space at the end
        df[col] = df[col].str.replace(r"( $)", "",regex=True)
        
        # Standardize the spelling of litres and ml
        df[col] = df[col].str.replace(r"(\s?l$|\s?lt$|\s?lt\.$|\sl\.$|\s?lts$|\s?lts\.$)", "l",regex=True)
        df[col] = df[col].str.replace(r"((?<![\w])|(?<=\d))(ml\.?$|cc\.?$)", "ml",regex=True)
        df[col] = df[col].str.replace(r"(\s?ml$)", "ml",regex=True)
        df[col] = df[col].str.replace(r"((?<=\d))(\s?l$)", "l",regex=True)

In [6]:
# Create copies of main columns to be modified
external_data["product_cleaned_from"]=external_data["product"].copy()
internal_data["product_cleaned_to"]=internal_data["product_name"].copy()

In [7]:
# Clean columns
string_processing(df=external_data,ignored_columns=["product"])
string_processing(df=internal_data,ignored_columns=["product_name"])

#### 4. String matching

In [8]:
# Create a vector of n-gram TFIDF frequencies
tfidf_vectorizer = TFIDF(n_gram_range=(3,3),min_similarity=0.8,clean_string=True)

# Define an instance of the PolyFuzz class using the tfidf_vectorizer model
model = PolyFuzz(tfidf_vectorizer)

from_data = list(external_data["product_cleaned_from"])
to_data = list(internal_data["product_cleaned_to"])

# Send the lists of elements for the matching
model.match(from_data, to_data)

# Obtain the matches
matches = model.get_matches()
matches.dropna(inplace=True)

# Drop duplicates
data_matched = matches.drop_duplicates(["From","To"]).reset_index(drop=True).copy()

In [9]:
# Add some additional similarity measures

# Token set ratio 
data_matched["Token_set_ratio"]=data_matched.apply(lambda x: fuzz.token_set_ratio(x["From"],x["To"])/100,axis=1)

# Levenshtein distance
data_matched["Edit_distance"]= data_matched.apply(lambda x: edit_distance(x['From'],x['To']),axis=1)

# Extract the unity of measures of the products
data_matched["Quantity_from"]=data_matched["From"].str.extract(r"(\d+?\.?\d+?\s?ml$|\d?\d+?\.?\d+?\s?l$|\s\d?l$)")
data_matched["Quantity_to"]=data_matched["To"].str.extract(r"(\d+?\.?\d+?\s?ml$|\d?\d+?\.?\d+?\s?l$|\s\d?l$)")

# Specify the origin columns used for the matching process to be able to match the dataframes
data_matched["Property_from"] = "product_cleaned_from"
data_matched["Property_to"] = "product_cleaned_to"

# Match the results with the input data to associate them with all their data
# First we merge the external_data
data_matched = pd.merge(data_matched,external_data,left_on="From",right_on=f"{data_matched.Property_from[0]}")

# Then we use the previous join to merge it with the internal data
final_results = pd.merge(data_matched,internal_data,left_on="To",right_on="product_cleaned_to")

final_results=final_results[["product","product_name","From","To","Similarity","Token_set_ratio","Edit_distance","Quantity_from","Quantity_to"]]

In [10]:
final_results

,product,product_name,From,To,Similarity,Token_set_ratio,Edit_distance,Quantity_from,Quantity_to
0,beverages coke diet vanilla 500ml,DIET VANILLA COKE X 500ML,coke diet vanilla 500ml,diet vanilla coke x 500ml,1.000000,1.0,12,500ml,500ml
1,beverages coke diet vanilla 500cc,DIET VANILLA COKE X 500ML,coke diet vanilla 500ml,diet vanilla coke x 500ml,1.000000,1.0,12,500ml,500ml
2,beverages coke diet vanilla 500 ml,DIET VANILLA COKE X 500ML,coke diet vanilla 500ml,diet vanilla coke x 500ml,1.000000,1.0,12,500ml,500ml
3,beverages coke diet vanilla 500 cc,DIET VANILLA COKE X 500ML,coke diet vanilla 500ml,diet vanilla coke x 500ml,1.000000,1.0,12,500ml,500ml
4,beverages coke diet vanilla 500 ml.,DIET VANILLA COKE X 500ML,coke diet vanilla 500ml,diet vanilla coke x 500ml,1.000000,1.0,12,500ml,500ml
...,...,...,...,...,...,...,...,...,...
3307,beverage chocolate pepsi 354ml,DIET CHOCOLATE PEPSI X 354ML,chocolate pepsi 354ml,diet chocolate pepsi x 354ml,0.953511,1.0,7,354ml,354ml
3308,beverage chocolate pepsi 354cc,DIET CHOCOLATE PEPSI X 354ML,chocolate pepsi 354ml,diet chocolate pepsi x 354ml,0.953511,1.0,7,354ml,354ml
3309,beverage chocolate pepsi 354 ml,DIET CHOCOLATE PEPSI X 354ML,chocolate pepsi 354ml,diet chocolate pepsi x 354ml,0.953511,1.0,7,354ml,354ml
3310,beverage chocolate pepsi 354 cc,DIET CHOCOLATE PEPSI X 354ML,chocolate pepsi 354ml,diet chocolate pepsi x 354ml,0.953511,1.0,7,354ml,354ml
